In [2]:
# Importar bibliotecas necessárias
import pandas as pd
from sqlalchemy import create_engine

# Configurações da conexão
db_config = {
    'user': 'practicum_student',
    'pwd': 'QnmDH8Sc2TQLvy2G3Vvh7',
    'host': 'yp-trainers-practicum.cluster-czs0gxyx2d8w.us-east-1.rds.amazonaws.com',
    'port': 5432,
    'db': 'data-analyst-final-project-db'
}

# Criar a string de conexão
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(
    db_config['user'],
    db_config['pwd'],
    db_config['host'],
    db_config['port'],
    db_config['db']
)

# Criar engine de conexão com o banco
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

# Exemplo de consulta SQL:
# query = 'SELECT * FROM books LIMIT 5;'
# pd.read_sql(query, con=engine)


## Objetivos do Estudo

Com o aumento do consumo de livros durante o período da pandemia, diversas startups passaram a explorar esse comportamento como oportunidade de negócio. O objetivo deste estudo é realizar uma análise exploratória e quantitativa sobre os dados de uma plataforma digital de leitura para:

- Entender o perfil do catálogo de livros disponível (volume, recência, qualidade);
- Identificar editoras e autores com maior influência ou engajamento;
- Avaliar o comportamento de avaliação dos usuários, incluindo padrões de uso intenso;
- Apoiar a tomada de decisão sobre o desenvolvimento de um novo produto digital voltado ao público leitor, com base em dados reais de consumo e preferências.

Todas as análises serão conduzidas diretamente sobre um banco de dados PostgreSQL utilizando consultas SQL otimizadas.


In [17]:
# Visualizar amostra da tabela books
query_books_amostra = 'SELECT * FROM books LIMIT 5;'
pd.read_sql(query_books_amostra, con=engine)


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [18]:
# Consulta SQL para contar livros publicados após 01/01/2000
query_livros_2000 = '''
SELECT COUNT(*) AS livros_pos_2000
FROM books
WHERE publication_date > '2000-01-01';
'''
resultado_livros_2000 = pd.read_sql(query_livros_2000, con=engine)
resultado_livros_2000


,livros_pos_2000
0,819


Livros lançados após 1º de janeiro de 2000

Resultado
Foram identificados **819 livros** publicados após 1º de janeiro de 2000.


In [19]:
# Amostra da tabela ratings
query_ratings_amostra = 'SELECT * FROM ratings LIMIT 5;'
pd.read_sql(query_ratings_amostra, con=engine)


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [20]:
# Consulta SQL: total de avaliações e média de classificação por livro
query_avaliacoes_livros = '''
SELECT 
    b.title AS titulo_livro,
    COUNT(r.rating_id) AS total_avaliacoes,
    ROUND(AVG(r.rating), 2) AS media_classificacao
FROM books b
JOIN ratings r ON b.book_id = r.book_id
GROUP BY b.title
ORDER BY total_avaliacoes DESC;
'''
resultado_avaliacoes = pd.read_sql(query_avaliacoes_livros, con=engine)
resultado_avaliacoes.head()


,titulo_livro,total_avaliacoes,media_classificacao
0,Twilight (Twilight #1),160,3.66
1,The Hobbit or There and Back Again,88,4.13
2,The Catcher in the Rye,86,3.83
3,Angels & Demons (Robert Langdon #1),84,3.68
4,Harry Potter and the Prisoner of Azkaban (Harr...,82,4.41


Os cinco livros com maior número de avaliações foram:

1. **Twilight (Twilight #1)** — 160 avaliações, média **3.66**
2. **The Hobbit or There and Back Again** — 88 avaliações, média **4.13**
3. **The Catcher in the Rye** — 86 avaliações, média **3.83**
4. **Angels & Demons (Robert Langdon #1)** — 84 avaliações, média **3.68**
5. **Harry Potter and the Prisoner of Azkaban** — 82 avaliações, média **4.41**


In [21]:
# Visualizar as primeiras linhas das tabelas
pd.read_sql('SELECT * FROM books LIMIT 5;', con=engine)
pd.read_sql('SELECT * FROM publishers LIMIT 5;', con=engine)


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [22]:
# Consulta SQL para identificar a editora com mais livros com mais de 50 páginas
query_editora_maior_producao = '''
SELECT 
    p.publisher AS editora,
    COUNT(b.book_id) AS total_livros
FROM books b
JOIN publishers p ON b.publisher_id = p.publisher_id
WHERE b.num_pages > 50
GROUP BY p.publisher
ORDER BY total_livros DESC
LIMIT 1;
'''
resultado_editora = pd.read_sql(query_editora_maior_producao, con=engine)
resultado_editora


,editora,total_livros
0,Penguin Books,42


Resultado
A editora que mais publicou livros com mais de 50 páginas foi:

- **Penguin Books** — com **42 livros completos** no catálogo.




In [23]:
# Visualizar amostras
pd.read_sql('SELECT * FROM books LIMIT 5;', con=engine)
pd.read_sql('SELECT * FROM authors LIMIT 5;', con=engine)
pd.read_sql('SELECT * FROM ratings LIMIT 5;', con=engine)


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [24]:
# Autor com maior média de avaliação, considerando apenas livros com pelo menos 50 avaliações
query_autor_top = '''
SELECT 
    a.author AS autor,
    ROUND(AVG(sub.avg_rating), 2) AS media_geral
FROM (
    SELECT 
        b.book_id,
        b.author_id,
        AVG(r.rating) AS avg_rating,
        COUNT(r.rating_id) AS total_avaliacoes
    FROM books b
    JOIN ratings r ON b.book_id = r.book_id
    GROUP BY b.book_id
    HAVING COUNT(r.rating_id) >= 50
) sub
JOIN authors a ON sub.author_id = a.author_id
GROUP BY a.author
ORDER BY media_geral DESC
LIMIT 1;
'''
resultado_autor = pd.read_sql(query_autor_top, con=engine)
resultado_autor


,autor,media_geral
0,J.K. Rowling/Mary GrandPré,4.28


Resultado
O autor com a maior média de classificação entre livros com pelo menos 50 avaliações foi:

- **J.K. Rowling / Mary GrandPré** — com média **4.28**.

In [25]:
# Visualizar amostra da tabela ratings
pd.read_sql('SELECT * FROM ratings LIMIT 5;', con=engine)


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [26]:
# Média de avaliações entre usuários que avaliaram mais de 50 livros
query_media_usuarios_ativos = '''
SELECT ROUND(AVG(avaliacoes), 2) AS media_avaliacoes
FROM (
    SELECT username, COUNT(*) AS avaliacoes
    FROM ratings
    GROUP BY username
    HAVING COUNT(*) > 50
) sub;
'''
resultado_usuarios_ativos = pd.read_sql(query_media_usuarios_ativos, con=engine)
resultado_usuarios_ativos


,media_avaliacoes
0,54.33


Resultado
Usuários que avaliaram mais de 50 livros registraram, em média, **54,33 avaliações cada**.



Conclusão Geral – Análise SQL

A análise SQL permitiu extrair informações estratégicas do banco de dados de uma plataforma digital de leitura, com foco em qualidade editorial, engajamento dos usuários e relevância de conteúdo. Abaixo, os principais insights extraídos das cinco consultas conduzidas:

1. **Atualidade do acervo**  
   Foram identificados **819 livros publicados após o ano 2000**, demonstrando que a plataforma mantém um catálogo contemporâneo, adequado às expectativas de leitores modernos.

2. **Livros com maior engajamento**  
   Obras como *Twilight* e *The Hobbit* lideram em número de avaliações, indicando títulos populares e com forte apelo entre os usuários. A média de avaliação desses livros confirma boa aceitação geral.

3. **Editoras com obras relevantes**  
   A **Penguin Books** foi a editora com maior número de livros com mais de 50 páginas, destacando-se como produtora de conteúdo denso e editorialmente relevante — um sinal positivo para curadoria e parcerias.

4. **Autores com alta reputação**  
   **J.K. Rowling / Mary GrandPré** apresentaram a maior média de avaliação entre livros com pelo menos 50 avaliações, reforçando seu posicionamento como autores de alto impacto qualitativo.

5. **Perfis de usuários altamente ativos**  
   Usuários que avaliaram mais de 50 livros realizaram, em média, **54,33 avaliações**, mostrando consistência de engajamento. Esse grupo representa uma base valiosa para iniciativas de fidelização, testes de novas funcionalidades e geração de conteúdo colaborativo.

